In [1]:
!pip install --q "snowflake-connector-python[pandas]"
!sudo pip install --q snowflake-ml-python==1.0.11 -U
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip install --q seaborn
!pip install --q plotly
!pip install --upgrade --q xgboost==1.7.3
!pip install --upgrade --q numpy==1.24.3
!pip install --upgrade --q pandas==1.5.3
!pip install --q statsmodels
!pip install --q fosforio

You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: snowflake-snowpark-python 1.21.1 has requirement cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0; python_version < "3.11", but you'll have cloudpickle 3.0.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'WHEEL'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'METADATA'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pi